In [1]:
# import section
import tensorflow as tf
import numpy as np
import PIL.Image as Image
import matplotlib.pyplot as plt

In [2]:
def add_image():
#     print(label)
#     print(img_name)
    image_path = 'data\dogs\\' + img_name
    img = Image.open(image_path)
    
    # convert to greyscale
#     img = img.convert('L')
    
    # resize
    img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)

    # Add to image_data array
    image_data.append([np.array(img), label])
    
    # Add flipped image to image_data array
#     image_data.append([np.array(img.transpose(Image.FLIP_LEFT_RIGHT)), label])
    

In [3]:
# Read data from file into a string
file = open("data\labels.csv", "r")
label_data = file.read()
file.close()

# Set Max breeds and Max Images per breed values
MAX_BREEDS = 10
MAX_IMAGES = 74 # 74 Minimum image count for first 10 breeds.
IMAGE_SIZE = 224
image_data = []

breeds = []
breed_full = False
breed_ctr = {}
# This alternative strategy will try to directly load images based on the names in the csv file.
label_entries = label_data.split("\n") 
for entry in label_entries[1:-1]: 
    name, label = entry.split(",")
    img_name = name + '.jpg'
    if not(breed_full) and not(label in breeds):
        add_image()
        breeds.append(label)
        breed_ctr[label] = 1
        if len(breeds) >= MAX_BREEDS:
            breed_full = True
    if label in breeds and breed_ctr[label] < MAX_IMAGES:
        add_image()
        breed_ctr[label] = breed_ctr[label] + 1

print(len(image_data))



734


In [4]:
# shuffle and split data
# import random
# random.shuffle(image_data)
# print(image_data[0][1])

#Set split percentage
TRAIN = .8

# fixes issues when the number of images do not reach the maximum limit.
if (len(image_data) < MAX_IMAGES * MAX_BREEDS):
    MAX_IMAGES = len(image_data)/ MAX_BREEDS

train_data = []
test_data = []

for breed in breeds:
    breed_train_ctr = 0
    for image in image_data:
        if image[1] == breed:
            if breed_train_ctr < MAX_IMAGES * TRAIN:
                breed_train_ctr+=1
                train_data.append(image)
            else:
                test_data.append(image)
            
# print(len(train_data))
# print(len(test_data))

train_images = []
train_labels = []

test_images = []
test_labels = []

for data in train_data:
    train_images.append(data[0])
    train_labels.append(breeds.index(data[1]))
    
for data in test_data:
    test_images.append(data[0])
    test_labels.append(breeds.index(data[1]))

# print((train_images[0]))
# print(train_labels[0])

train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

print(train_images.shape)
print(test_images.shape)

## This code is needed if images are converted to greyscale ##
# train_images = train_images.reshape((train_images.shape[0], train_images.shape[1], train_images.shape[2], 1))
# test_images = test_images.reshape((test_images.shape[0], test_images.shape[1], test_images.shape[2],1))

(590, 224, 224, 3)
(144, 224, 224, 3)


In [5]:
# from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from time import time
from tensorflow.python.keras.callbacks import TensorBoard, EarlyStopping

# Parameters


# Build CNN

model = Sequential()
model.add(Conv2D(64, kernel_size = (3,3), strides=(1,1), activation='relu', input_shape=(IMAGE_SIZE,IMAGE_SIZE,3)))
model.add(Conv2D(64, kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
# model.add(Dropout(0.33))

model.add(Conv2D(128, kernel_size = (3,3), activation='relu'))
model.add(Conv2D(128, kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
# model.add(Dropout(0.33))

model.add(Conv2D(256, kernel_size = (3,3), activation='relu'))
model.add(Conv2D(256, kernel_size = (3,3), activation='relu'))
model.add(Conv2D(256, kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
# # model.add(Dropout(0.33))

model.add(Conv2D(512, kernel_size = (3,3), activation='relu'))
model.add(Conv2D(512, kernel_size = (3,3), activation='relu'))
model.add(Conv2D(512, kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size = (3,3), activation='relu'))
model.add(Conv2D(512, kernel_size = (3,3), activation='relu'))
model.add(Conv2D(512, kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(rate = 0.50))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.33))
model.add(Dense(MAX_BREEDS, activation = 'softmax'))

tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))

model.compile(optimizer='adam',
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
model.fit(train_images, train_labels, batch_size = 50, epochs = 10000, verbose = 1, callbacks=[tensorboard, es], validation_data=(test_images, test_labels))

Train on 590 samples, validate on 144 samples
Epoch 1/10000
590/590 [==============================] - 28s 48ms/sample - loss: 2.6976 - acc: 0.1119 - val_loss: 13.9032 - val_acc: 0.1319
Epoch 2/10000


ResourceExhaustedError: OOM when allocating tensor with shape[50,64,222,222] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/Adam/gradients/conv2d_1/Conv2D_grad/Conv2DBackpropInput}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
# print(test_images.shape)
# print(test_labels.shape)

loss, acc = model.evaluate(test_images, test_labels, verbose=0)
print(acc)
print(loss)

In [ ]:
# Beat: 42.1% on 10
# 36% with 2x the nodes